In [21]:
import dsw_python
import os
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
import edge_boxes_python
import selectivesearch

In [22]:
# Paths for Kitti Object detection files
kittiLeftImagePath = "/scratch/fs2/KITTI/data_object_image_2/training/image_2"
kittiRightImagePath = "/scratch/fs2/KITTI/data_object_image_3/training/image_3"
kittiCalibPath = "/scratch/fs2/KITTI/data_object_calib/training/calib"
kittiLabelPath = "/scratch/fs2/KITTI/training/label_2"
kittiDispPath = "/scratch/fs2/KITTI/disparities"

In [28]:
class Label():
    def __init__(self, **entries):
        self.x = 0
        self.y = 0
        self.w = 0
        self.h = 0
        self.l_type = ""
        self.occlusion = 0
        self.best_overlap = 0.
        self.ignore = -1.
        if entries != None:
            self.__dict__.update(entries)
        
class Hyp():
    def __init__(self, **entries):
        self.x = 0
        self.y = 0
        self.w = 0
        self.h = 0
        
        if entries != None:
            self.__dict__.update(entries)
        
class Calib():
    def __init__(self):
        self.p_left = np.zeros((3,4))        
        self.p_right = np.zeros((3,4))
        self.k_left = np.zeros((3,3))
        self.r0 = np.zeros((3,3))
        self.tx = 0.
        
class EvalTrainingSS():
    def __init__(self):
        self.frames = []
        self.sigma = 0.
        self.min_size = 0
        self.scale = 0
        
class EvalSS():
    def __init__(self, min_size, k):
        self.min_size_ = min_size
        self.k_ = k
        self.frames = []
        
    def detect(self, img):
        img_lbl, regions = selectivesearch.selective_search(img, scale=self.k_, sigma=0.9, min_size=self.min_size_)
        hyp_list = []

        for r in regions:
            h = Hyp()
            h.x, h.y, h.w, h.h = r['rect']
            hyp_list.append(h)           
        return hyp_list
    
    def save_results(self, save_dir):
        save_list = []
        save_list.append(dict(k = self.k_))
        with open(save_dir + '/result_SS_k_' + str(self.k_) + '.json', 'w') as output:
            for frame in self.frames:
                data = {}
                data['frame'] = dict(labels = [ob.__dict__ for ob in frame.labels], hyps = [ob.__dict__ for ob in frame.hyps])
                save_list.append(data)
            json.dump(save_list , output)

        
class EvalEB():
    def __init__(self, gamma):
        self.eb_ = edge_boxes_python.edge_boxes_python('/home/muejul3/git_repos/edges/cpp/external/gop_1.3/data/sf.dat')
        self.gamma_ = gamma
        self.frames = []

        
    def detect(self, img):
        bbs = self.eb_.get_edge_boxes(img, self.gamma_)
        hyp_list = []
        for bb in bbs:
            h = Hyp()
            h.x = int(bb[0])
            h.y = int(bb[1])
            h.w = int(bb[2])
            h.h = int(bb[3])
            hyp_list.append(h)
        return hyp_list
     
    def save_results(self, save_dir):
        save_list = []
        save_list.append(dict(gamma = self.gamma_))
        with open(save_dir + '/result_EB_gamma_' + str(self.gamma_) + '.json', 'w') as output:
            for frame in self.frames:
                data = {}
                data['frame'] = dict(labels = [ob.__dict__ for ob in frame.labels], hyps = [ob.__dict__ for ob in frame.hyps])
                save_list.append(data)
            json.dump(save_list , output)
        
class EvalDSW():
    
    def __init__(self, world_width=None, world_height=None, hyp_aspect=None, 
                 hyp_min_width=None, hyp_max_width=None, max_stddev=None, 
                 step_perc=None, method = 1, hyp_class_id = 10, max_nans = 6):
        
        self.frames = []
        self.world_width_ = world_width
        self.world_height_ = world_height
        self.hyp_aspect_ = hyp_aspect
        self.hyp_min_width_ = hyp_min_width
        self.hyp_max_width_ = hyp_max_width
        self.hyp_class_id_ = hyp_class_id
        self.max_nans_ = max_nans
        self.max_stddev_ = max_stddev
        self.step_perc_ = step_perc
        self.method_ = method
        self.min_disp_ = 0.
        self.max_disp_ = 0.
        self.step_disp_ = 0.

        # Init DSW
        if world_width != None and world_height != None:
            self.init_dsw()
 
    def set_disparity_parameters(self, min_disp, max_disp, step_disp):
        self.min_disp_ = min_disp
        self.max_disp_ = max_disp
        self.step_disp_ = step_disp
        
    def calc_lut(self, calib):
        
        if self.max_disp_ == 0.0 or self.step_disp_ == 0.0:
            print "ERROR: use set_disparity_parameters before: max_disp or step_disp are zero"
        self.dsw_.initLookUpTable(calib.tx, calib.k_left, np.zeros((5,1)), self.min_disp_, self.max_disp_, self.step_disp_)        
    
    def init_dsw(self):
        self.dsw_ = dsw_python.DisparitySlidingWindow(self.world_width_, self.world_height_, self.hyp_aspect_, self.hyp_min_width_, self.hyp_max_width_, self.hyp_class_id_, self.max_nans_, self.max_stddev_, self.step_perc_, self.method_)
    
    def detect(self, img_disp, tx):
        rect_list = []
        hyp_list = []
        rect_list = self.dsw_.generate_py(img_disp, tx)
        for rect in rect_list:
            h= Hyp()
            h.x = rect[0]
            h.y = rect[1]
            h.w = rect[2] 
            h.h = rect[3]
            hyp_list.append(h)
        return hyp_list
    
    def save_results(self, save_dir):
        save_list = []
        save_list.append(dict(stddev = self.max_stddev_, step_perc = self.step_perc_))
        with open(save_dir + '/result_dsw_sigma_' + str(self.max_stddev_) + '_step_' + str(self.step_perc_) + '.json', 'w') as output:
            for frame in self.frames:
                data = {}
                data['frame'] = dict(labels = [ob.__dict__ for ob in frame.labels], hyps = [ob.__dict__ for ob in frame.hyps])
                save_list.append(data)
            json.dump(save_list , output)
        
        
class EvalFrame():
    def __init__(self):
        self.labels = []
        self.hyps = []
              
class iouResults():
    def __init__(self):
        self.iou_list = []
        self.tp_rate = []
        self.stddev = 0.
        self.step_perc = 0.
        self.hyps = 0
        
class iouResultsSS():
    def __init__(self):
        self.iou_list = []
        self.tp_rate = []
        self.k = 0.
        self.hyps = 0

In [29]:
# Empty lists for images and calib
leftImgPathList = []
rightImgPathList = []
calibPathList = []
labelPathList = []
dispPathList = []

# Get list of all left image paths
for file in sorted(os.listdir(kittiLeftImagePath)):
    if file.endswith(".png"):
        leftImgPathList.append(os.path.join(kittiLeftImagePath,file))
        
# Get list of all right image paths
for file in sorted(os.listdir(kittiRightImagePath)):
    if file.endswith(".png"):
        rightImgPathList.append(os.path.join(kittiRightImagePath,file))

# Get list of all calib paths
for file in sorted(os.listdir(kittiCalibPath)):
    if file.endswith(".txt"):
        calibPathList.append(os.path.join(kittiCalibPath,file))
        
# Get list of all label paths
for file in sorted(os.listdir(kittiLabelPath)):
    if file.endswith(".txt"):
        labelPathList.append(os.path.join(kittiLabelPath,file)) 
        
# Get list of all disparity images
for file in sorted(os.listdir(kittiDispPath)):
    if file.endswith(".png"):
        dispPathList.append(os.path.join(kittiDispPath,file))

if not len(leftImgPathList) == len(rightImgPathList) == len(calibPathList) == len(labelPathList) == len(dispPathList):
    print "ERROR: Number of left/right/disparity images and calib/label files should be the same. Found the following:"
    print "Number of left images: " + str(len(leftImgPathList))
    print "Number of right images: " + str(len(rightImgPathList))
    print "number of disparity images: " + str(len(dispPathList))
    print "number of calib files: " + str(len(calibPathList))
    print "number of label files: " + str(len(labelPathList))

In [30]:
def readKittiLabels(path):
    label_list = []
    with open(path) as f:
        
        for line in f:
            l = Label()
            lines = line.split()
            for idx, token in enumerate(lines):

            # read first two tokens separately
                if idx == 0:
                    l.l_type = str(token)
                elif idx == 2:
                    l.occlusion = int(token)
                elif idx == 4:
                    l.x = int(float(token))
                    l.y = int(float(lines[idx+1]))
                    l.w = int(float(lines[idx+2])) - int(float(token))
                    l.h = int(float(lines[idx+3])) - int(float(lines[idx+1]))
                    break
            label_list.append(l)
    return label_list

def readKittiCalib(path):
    
    calib = Calib()
    with open(path) as f:

        for line in f:
            l = Label()
            lines = line.split()
            vector = np.asarray(lines[1:],dtype=np.float32)
            if len(lines) == 0:
                continue
                   
            if lines[0] == "P2:":
                calib.p_left = vector.reshape(3,4)
            elif lines[0] == "P3:":  
                calib.p_right = vector.reshape(3,4)
            elif lines[0] == "R0_rect:":
                calib.r0 = vector.reshape(3,3)
    
    calib.tx = float(calib.p_right[0,3])
    rot = np.zeros((3,3))
    trans = np.zeros((4,1))
    cam_matrix = np.zeros((3,3),dtype= np.double)
    cam_matrix = cv2.decomposeProjectionMatrix(calib.p_left, cam_matrix, rot, trans)
    calib.k_left = cam_matrix[0]
    return calib

def union( a, b):

    x = min(a.x, b.x)
    y = min(a.y, b.y)
    w = max(a.x + a.w, b.x + b.w) - x
    h = max(a.y + a.h, b.y + b.h) - y
    return w * h

def intersection( a, b):

    x = max(a.x, b.x)
    y = max(a.y, b.y)
    w = min(a.x + a.w, b.x + b.w) - x
    h = min(a.y + a.h, b.y + b.h) - y
    if w < 0 or h < 0:
      return 0.0  # or (0,0,0,0) ?
    return w * h

def bb_intersection_over_union( label, hyp):

    # intersection
    num = float(intersection(label, hyp))
    # union
    den = float(union(label, hyp))
    # intersection/union
    iou = num / den

    # return the intersection over union value
    return iou              

In [47]:
# evaluate method
# choose method
method = "SELECTIVESEARCH"

if method == "DSW":
    
    # Params of DSW
    dsw_scale = 0.7
    dsw_min_size = 100
    objectWidth = 0.6
    objectHeight = 1.73
    hypAspect = 2.88
    hypMinWidth = 10
    hypMaxWidth = 200
    hypClassId = 10
    maxNans = 6
    maxStddev = 0.01
    stepPerc = 0.3
    homogeneity_method = 1
    min_disparity = 0
    max_disparity = 115
    step_disparity = 1./16.
    save_dir = '/home/muejul3/git_repos/disparity-sliding-window/results/results_dsw'
    
    # Init DSW
    eval_training = EvalDSW(objectWidth, objectHeight, hypAspect, hypMinWidth, hypMaxWidth, maxStddev, stepPerc, homogeneity_method, hypClassId, maxNans)
    eval_training.set_disparity_parameters(min_disparity, max_disparity, step_disparity)
elif "EDGEBOXES" in method:
    gamma = 0.5
    save_dir = '/home/muejul3/git_repos/disparity-sliding-window/results/results_eb'
    eval_training = EvalEB(gamma)
      
elif method == "SELECTIVESEARCH":
    min_size = 100
    k = 100
    save_dir = '/home/muejul3/git_repos/disparity-sliding-window/results/results_ss'
    eval_training = EvalSS(min_size, k)
else:
    print "ERROR: Method not known!"


In [48]:
# for all images in KITTI

# SGM parameters
minDisparity = 2
numDisparities = 114 - minDisparity
windowSize = 5
p1 = 8*3*windowSize**2
p2 = 32*3*windowSize**2
dispMaxDiff = 1
preFilterCap = 0
uniqueRatio = 10
speckleWindowSize = 100
speckeRange = 32
fullDP = False


# SGM Constructor
sgm = cv2.StereoSGBM(minDisparity, numDisparities, windowSize, p1, p2, dispMaxDiff, preFilterCap, uniqueRatio, speckleWindowSize,   speckeRange, fullDP)

for label_path, left_img_path, right_img_path, calib_path, disp_path in zip(labelPathList, leftImgPathList, rightImgPathList, calibPathList, dispPathList):

    # Read KITTI labels
    label_list = readKittiLabels(label_path)

    ped_in_frame = False

    # Check if pedestrian is in frame
    for l in label_list:
        if l.l_type == "Pedestrian":
            ped_in_frame = True

    # We only evaulate Pedestrians
    if ped_in_frame:
        
        # New Evaluate Frame
        eval_frame = EvalFrame()
        eval_frame.labels = label_list
        
        if method == "DSW":
            # Read Calibration
            calib = readKittiCalib(calib_path)
        
            # Recalculate LookupTable
            eval_training.calc_lut(calib)

            #img_disp = cv2.imread(disp_path, -1)
            #disparity = img_disp.astype(np.float32) /16.0

            img_left = cv2.imread(left_img_path, -1)
            img_right = cv2.imread(right_img_path, -1)
            disparity = sgm.compute(img_left, img_right)
            disparity = disparity.astype(np.float32) /16.0
            disparity[disparity==1.0]=np.nan
            rect_list = eval_training.detect(disparity, calib.tx)
            eval_frame.hyps.extend(rect_list)
        elif method == "EDGEBOXES":            
            img_left = cv2.imread(left_img_path, -1)
            rect_list = eval_training.detect(img_left)
            eval_frame.hyps.extend(rect_list)
        elif method == "SELECTIVESEARCH" :
            img_left = cv2.imread(left_img_path, -1)
            rect_list = eval_training.detect(img_left)
            eval_frame.hyps.extend(rect_list)
        eval_training.frames.append(eval_frame)
eval_training.save_results(save_dir)

KeyboardInterrupt: 

813


In [45]:
############## Calculate ROC Curves for Proposal Methods ##################
result_dir = '/home/muejul3/git_repos/disparity-sliding-window/results/results_dsw/Untitled Folder/'
result_extension = ".json"

# keys in json
frame_key = 'frame'
hyps_key = 'hyps'
labels_key = 'labels'
label_type_key = 'l_type'
stddev_key = 'stddev'
step_perc_key = 'step_perc'

# eval_class
eval_class = "Pedestrian"

# check extension
assert result_extension == ".json", "Only extension .json is actually supported, your extension is: " + str(result_extension)

eval_all = []
# find all result files with extension
for file in os.listdir(result_dir):
    hyps_nr = 0
    if file.endswith(result_extension):
        eval_dsw = EvalDSW()
        # open file
        with open(result_dir + file, 'r') as fp:
            eval = json.load(fp)           
            for elem in eval:
                
                # saved frames
                if frame_key in elem.keys():
                    eval_frame = EvalFrame()                
                    #evaluate all labels
                    hyps_nr += len(elem[frame_key][hyps_key])
                    for label in elem[frame_key][labels_key]:
                        l = Label(**label)
                        if label[label_type_key] == eval_class:
                            l.ignore = -1.0
                            best_ov = 0.
                            for hyp in elem[frame_key][hyps_key]:
                                h = Hyp(**hyp)
                                iou = bb_intersection_over_union(l,h)
                                if iou > best_ov:
                                    best_ov = iou                   
                            l.best_overlap = best_ov
                            eval_frame.labels.append(l)
                        else:
                            l.ignore = 1.0
                elif step_perc_key in elem.keys() or stddev_key in elem.keys():
                    eval_dsw.step_perc_ = elem[step_perc_key]
                    eval_dsw.stddev_ = elem[stddev_key]
                        
                eval_dsw.frames.append(eval_frame)
        eval_all.append(eval_dsw)
    print file
    print "Average Number of proposals: " + str(float(hyps_nr)/float(len(eval)))




result_SS_k_800.json
Average Number of proposals: 736.097752809


In [46]:
# Plot Options   
plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}"]
params = {'text.usetex' : True,
          'font.size' : 20,
          'font.family' : 'lmodern',
          'text.latex.unicode': True,
          }

plt.rcParams.update(params) 
fig_all = plt.figure()
ax_all = fig_all.gca()

### calculate recall(IoU) curve and plot everything ###
iou_list = np.arange(0.0, 1.05, 0.05)
for train in eval_all:
    tp_rate_list = []
    for iou in iou_list:
        tp_count = 0
        label_count = 0
        for frame in train.frames:
            for l in frame.labels:
                if l.l_type == "Pedestrian":
                    label_count +=1
                    if l.best_overlap >= iou:
                        tp_count +=1
        tp_rate_list.append(float(tp_count)/float(label_count))
    ylab = 'recall'
    label = 'DSW: $\Delta=' + str(train.step_perc_) + ',\sigma=' + str(train.stddev_) + '$' 
    alpha = 0.8
    ax_all.set_xlabel('$IoU$', fontsize = 20)
    ax_all.set_ylabel(ylab, fontsize = 20)
    ax_all.set_ylim([0.01, 1.0])
    ax_all.set_xlim([0.0, 1.0])
    ax_all.set_xticks(np.arange(0.0, 1.1, 0.1))

    ax_all.plot(iou_list, tp_rate_list, alpha=alpha, label=label, linestyle='-')
ax_all.legend(loc=4, prop={'size': 14})
      

fig_all.tight_layout()
fig_all.show()    

AttributeError: EvalDSW instance has no attribute 'stddev_'

In [38]:


#fig_all.savefig(plot_dir + "w_min=" + str(min_width) + "roc_all.eps")